In [1]:
pip install gekko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 47.0 MB/s eta 0:00:00


In [2]:
from gekko import GEKKO

In [3]:
model = GEKKO()
model.options.SOLVER=1

model.solver_options = ['minlp_maximum_iterations 500', \
                    
                    'minlp_max_iter_with_int_sol 10', \
                    
                    'nlp_maximum_iterations 50', \
                    
                    'minlp_integer_tol 0.05', \
                    
                    'minlp_gap_tol 0.01']

#Definig the decision variables
x1 = model.Var(lb=0,ub=20,integer=True)
x2 = model.Var(lb=0,ub=20,integer=True)
x3 = model.Var(lb=0,ub=20,integer=True)


#Defining the objective function
model.Minimize((10*((x1 - 4)**3)) + ((x2 - 7)**3) + (20*((x3 - 2)**3)))


#Defining the comstraints
model.Equation(x1+x2+x3==20)

In [4]:
model.solve(disp=False)
milp_obj = model.options.objfcnval

In [5]:
print('The value of x1 is ', x1.value)
print('The value of x2 is ',  x2.value)
print('The value of x3 is ', x3.value)
print('The Objective value is as follows ', model.options.objfcnval)

The value of x1 is  [5.0]
The value of x2 is  [12.0]
The value of x3 is  [3.0]
The Objective value is as follows  155.0



$x_1 = 5$, $x_2 = 12$, $x_3 = 3$
with objective value = $155$

In [6]:
!pip install -q pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.7 MB/s eta 0:00:00


In [7]:
import numpy as np
from pyomo.environ import * 

In [8]:
ls = [[], [], []]
for i in range(0, 21, 1):
  ls[0].append(10*(i-4)**3)
  ls[1].append((i-7)**3)
  ls[2].append( 20*(i-2)**3)

In [9]:
model_ = ConcreteModel()

In [10]:
model_.lmda1 = Var(range(len(ls[0])),domain=NonNegativeReals, bounds=(0,1))
model_.lmda2 = Var(range(len(ls[0])),domain=NonNegativeReals, bounds=(0,1))
model_.lmda3 = Var(range(len(ls[0])),domain=NonNegativeReals, bounds=(0,1))
model_.x1 = Var(range(len(ls[0])),domain=Binary)
model_.x2 = Var(range(len(ls[0])),domain=Binary)
model_.x3 = Var(range(len(ls[0])),domain=Binary)

In [11]:
model_.obj = Objective(expr=summation(ls[0], model_.lmda1)+summation(ls[1], model_.lmda2)+summation(ls[2], model_.lmda3),sense=minimize)

In [12]:
model_.constraints = ConstraintList()

In [13]:
model_.constraints.add(summation(model_.lmda1) == 1)
model_.constraints.add(summation(model_.lmda2) == 1)
model_.constraints.add(summation(model_.lmda3) == 1)
model_.constraints.add(summation([i for i in range(0,21)], model_.lmda1) + summation([i for i in range(0,21)], model_.lmda2) + summation([i for i in range(0,21)], model_.lmda3) == 20)

In [14]:
for i in range(len(ls[0])):
  model_.constraints.add( model_.lmda1[i] - model_.x1[i] <= 0)
  model_.constraints.add( model_.lmda2[i] - model_.x2[i] <= 0)
  model_.constraints.add( model_.lmda3[i] - model_.x3[i] <= 0)

In [15]:
temp_ls = []
for i in range(len(ls[0])):
  for j in range(len(ls[0])):
    if i!=j-1 and i!=j+1 and i!=j:
      if (j,i) in temp_ls:
        pass
      else:
        temp_ls.append((i,j))

In [16]:
for index in temp_ls:
  model_.constraints.add(model_.x1[index[0]] + model_.x1[index[1]] <= 1)
  model_.constraints.add(model_.x2[index[0]] + model_.x2[index[1]] <= 1)
  model_.constraints.add(model_.x3[index[0]] + model_.x3[index[1]] <= 1)

In [17]:
model_.constraints.add(summation(model_.x1) <= 2)
model_.constraints.add(summation(model_.x2) <= 2)
model_.constraints.add(summation(model_.x3) <= 2)

In [18]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.11.4+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.108.6+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-2_amd64.deb ...
Unpacking coinor-libcgl1 (0.60.3+repack1-2) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.10.3+repack1-1build1_amd64.deb ...
Unpacking coinor-libcbc3 (2.10.3+repack1-1build1) ...
Sel

In [19]:
opt_cbc = SolverFactory('cbc')

In [20]:
result = opt_cbc.solve(model_)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [21]:
print('\nObjective = ', model_.obj())


Objective =  155.0


We can see that the approximate objective value is identical to the nonlinear model's ideal objective value, hence the error % is 0.